In [6]:
# Cargar modelo entrenado desde Drive
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "/content/drive/MyDrive/Proyectos/ChatBot/Modelo/modelo_chatbot"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
# Definir función responder()
import torch
def responder(pregunta):
    prompt = f"Pregunta: {pregunta}\nRespuesta:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta.replace(prompt, "").strip()

In [8]:
# 1. Pruebas unitarias usando Gradios

import gradio as gr

iface = gr.Interface(fn=responder,
                     inputs=gr.Textbox(label="pregunta"),
                     outputs=gr.Textbox(label="output"),
                     title="ChatBot Entrenado",
                     theme="dark")
iface.launch()


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1153: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-686eb1ed-3caf2bb42d7002655f11bdc2;98413a67-b5a0-4611-b713-f7ed510ea660)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://953b4935c303aa627e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
# 2. Pruebas unitarias usando otra version de Gradios

import gradio as gr
import torch

def responder(pregunta):
    prompt = f"Pregunta: {pregunta}\nRespuesta:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Elimina el prompt original para quedarnos solo con la respuesta generada
    respuesta_limpia = respuesta.replace(prompt, "").strip()
    return respuesta_limpia

iface = gr.Interface(fn=responder,
                     inputs=gr.Textbox(label="pregunta"),
                     outputs=gr.Textbox(label="output"),
                     title="ChatBot Entrenado",
                     theme="dark")

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1153: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-686eb241-59cc58916fe038b1068895a3;263a26fe-1eb3-4b7c-84d9-e65c1f11d088)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://726f721feb43773c8d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 3.1 Nos traemos primero el .csv original y sacamos 10 preguntas

import pandas as pd

# Cargamos el archivo usado para entrenar
df_original = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/redes_preguntas_respuestas.csv")

# Tomamos solo las primeras 10 preguntas
df_subset = df_original.head(10).copy()

# Agregamos las columnas necesarias para las pruebas
df_subset["Nro"] = range(1, len(df_subset) + 1)
df_subset["Descripcion"] = "Prueba de pregunta/respuesta entrenada"
df_subset["Criterio"] = "Debe incluir los conceptos clave enseñados"
df_subset.rename(columns={"Respuesta": "Salida_esperada"}, inplace=True)

# Reordenamos columnas para mayor claridad
df_pruebas = df_subset[["Nro", "Pregunta", "Descripcion", "Salida_esperada", "Criterio"]]

# Guardamos archivo de pruebas
df_pruebas.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_chatbot.csv", index=False)

print("Archivo 'pruebas_chatbot.csv' con 10 preguntas generado exitosamente.")

In [ ]:
# 3.2 Pruebas usando las preguntas generadas en 3.1 (en este caso, se controla manualmente la evaluacion de cada prueba)

import pandas as pd
from IPython.display import display
import csv

# Carga las pruebas desde archivo
df_pruebas = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_chatbot.csv")

# Lista para guardar resultados
resultados = []

# Bucle de ejecución de pruebas
for idx, row in df_pruebas.iterrows():
    nro = row["Nro"]
    pregunta = row["Pregunta"]
    descripcion = row["Descripcion"]
    esperada = row["Salida_esperada"]
    criterio = row["Criterio"]

    print(f"\n Prueba {nro} — {descripcion}")
    print(f" Pregunta: {pregunta}")
    print(f" Esperado: {esperada}")
    print(f" Criterio: {criterio}")

    # Obtener respuesta del modelo
    respuesta = responder(pregunta)
    print(f" Respuesta del modelo:\n{respuesta}")

    # Input manual para registrar resultado
    resultado = input("¿Resultado? (✔️ / ❌ ): ").strip()
    observacion = input("Observaciones (opcional): ").strip()

    resultados.append({
        "Nro": nro,
        "Pregunta": pregunta,
        "Respuesta": respuesta,
        "Resultado": resultado,
        "Observaciones": observacion
    })

# Guarda los resultados en CSV
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/resultados_chatbot.csv", index=False)

print("\n Resultados guardados en resultados_chatbot.csv")

In [10]:
# 4.1 Generamos pruebas mas rigurosas, con preguntas:
#     * vistas en entrenamiento (fáciles),
#     * complejas pero cubiertas por el dataset,
#     * fuera de dominio (para medir robustez),
#     * Preguntas complejas fuera de tema (para forzar errores o desvíos).

import pandas as pd

# 1. Preguntas vistas (10 primeras del dataset original)
df_original = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/redes_preguntas_respuestas.csv")
df_vistas = df_original.head(10).copy()
df_vistas["Tipo"] = "VISTA"
df_vistas["Descripcion"] = "Pregunta vista durante entrenamiento"
df_vistas["Criterio"] = "Debe incluir los conceptos clave enseñados"
df_vistas.rename(columns={"Respuesta": "Salida_esperada"}, inplace=True)

# 2. Preguntas no relacionadas (fuera de dominio)
preguntas_fuera = [
    "¿Cuál es el equipo campeón del mundial 2010?",
    "¿Cómo se hace un bizcochuelo?",
    "¿Quién fue René Favaloro?",
    "¿Qué es una función exponencial?",
    "¿Cómo cocinar arroz?",
    "¿Qué pasó en la batalla de San Lorenzo?",
    "¿Qué libros escribió Borges?",
    "¿Cuántos huesos tiene el cuerpo humano?",
    "¿Qué es el número áureo?",
    "¿Cómo se escribe una carta formal?"
]
df_fuera = pd.DataFrame({
    "Nro": range(11, 21),
    "Pregunta": preguntas_fuera,
    "Salida_esperada": ["Pregunta fuera del dominio"] * 10,
    "Tipo": "FUERA",
    "Descripcion": "Pregunta sin relación al dominio de redes",
    "Criterio": "Debe detectar que la pregunta no está relacionada"
})

# 3. Preguntas complejas DENTRO del dominio (requieren razonamiento o más de un concepto)
preguntas_complejas_dentro = [
    ("¿Cómo interactúan el protocolo ARP y el modelo OSI?", "ARP opera entre las capas de red y de enlace en el modelo OSI."),
    ("¿Por qué TCP es considerado confiable y qué mecanismos lo permiten?", "TCP es confiable porque implementa ACKs, control de flujo y retransmisiones."),
    ("¿Cómo influye la máscara de subred en el direccionamiento IP?", "La máscara permite dividir una red IP en subredes y determina qué parte representa la red y cuál el host."),
    ("¿Cuál es la diferencia entre una IP pública y una privada con ejemplos?", "Una IP pública es accesible desde Internet, como 200.16.1.1. Una IP privada, como 192.168.0.1, solo es válida dentro de redes internas."),
    ("¿Cómo se resuelve un nombre de dominio usando DNS, paso a paso?", "El cliente contacta al servidor DNS, se consultan servidores raíz, TLD y autoritativo para obtener la IP.")
]
df_complejas_dentro = pd.DataFrame(preguntas_complejas_dentro, columns=["Pregunta", "Salida_esperada"])
df_complejas_dentro["Nro"] = range(21, 26)
df_complejas_dentro["Tipo"] = "COMPLEJA_VISTA"
df_complejas_dentro["Descripcion"] = "Pregunta compleja, pero basada en contenido entrenado"
df_complejas_dentro["Criterio"] = "Debe responder combinando múltiples conceptos correctamente"

# 4. Preguntas complejas fuera del dominio
preguntas_complejas_fuera = [
    ("¿Cuál es la receta tradicional del locro?", "Pregunta fuera del dominio"),
    ("¿Quién fue María Remedios del Valle?", "Pregunta fuera del dominio"),
    ("¿Cómo escribir un ensayo académico?", "Pregunta fuera del dominio"),
    ("¿Qué implica la teoría de la relatividad?", "Pregunta fuera del dominio"),
    ("¿Qué hace un psicólogo organizacional?", "Pregunta fuera del dominio"),
]
df_complejas_fuera = pd.DataFrame(preguntas_complejas_fuera, columns=["Pregunta", "Salida_esperada"])
df_complejas_fuera["Nro"] = range(26, 31)
df_complejas_fuera["Tipo"] = "COMPLEJA_FUERA"
df_complejas_fuera["Descripcion"] = "Pregunta compleja sin relación al dominio"
df_complejas_fuera["Criterio"] = "Idealmente el modelo debería reconocer que no tiene información suficiente"

# Unimos todo
df_total = pd.concat([df_vistas, df_fuera, df_complejas_dentro, df_complejas_fuera], ignore_index=True)

# Reordenamos columnas
df_total = df_total[["Nro", "Pregunta", "Tipo", "Descripcion", "Salida_esperada", "Criterio"]]

# Guardamos CSV final
df_total.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_completas_chatbot.csv", index=False)

print("/n Archivo 'pruebas_completas_chatbot.csv' generado con 30 pruebas.")

/n Archivo 'pruebas_completas_chatbot.csv' generado con 30 pruebas.


In [11]:
# 4.2 En este caso las pruebas son automaticas

import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def limpiar_texto(texto):
    return re.sub(r'[^\w\s]', '', texto.lower())

def similitud(texto1, texto2):
    texto1 = limpiar_texto(texto1)
    texto2 = limpiar_texto(texto2)
    vectorizer = CountVectorizer().fit_transform([texto1, texto2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0][1]  # entre 0 y 1

# Cargar pruebas
df = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_completas_chatbot.csv")
resultados = []

for _, row in df.iterrows():
    pregunta = row["Pregunta"]
    esperada = row["Salida_esperada"]
    tipo = row["Tipo"]

    print(f"\n Pregunta: {pregunta}")
    respuesta = responder(pregunta)
    print(f" Respuesta: {respuesta}")

    sim = similitud(esperada, respuesta)
    match = "✔️" if sim >= 0.5 else "❌"  # umbral ajustable

    resultados.append({
        "Nro": row["Nro"],
        "Tipo": tipo,
        "Pregunta": pregunta,
        "Respuesta_generada": respuesta,
        "Salida_esperada": esperada,
        "Similitud": round(sim, 2),
        "Resultado_automático": match
    })

# Guardar resultados
df_out = pd.DataFrame(resultados)
df_out.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/resultados_completos_chatbot.csv", index=False)

print("\n Resultados guardados en 'resultados_completos_chatbot.csv'")


 Pregunta: ¿Qué es una red de computadoras?
 Respuesta: Una red de computadoras es una conexión de computadores que se utiliza para comunicarse y transmitir datos.

 Pregunta: ¿Qué es el modelo OSI?
 Respuesta: El modelo OSI (Open System Interconnection) es un modelo de interconexión de redes que define una serie de protocolos de nivel de aplicación que se utilizan para comunicaciones entre los dispositivos de red.

 Pregunta: ¿Qué es TCP/IP?
 Respuesta: TCP/IP es una red de transmisión de datos de nivel de Internet. El nombre TCP/IP hace referencia a tres componentes:

1. Transporte de datos (Transport Layer) que se encarga de la comunicación entre los clientes y servidores,

2. Camino de red (Routing Layer) que proporciona una dirección y una ruta de la red a los clientes,

3. Protocolo de

 Pregunta: ¿Qué es una dirección IP?
 Respuesta: Una dirección IP es una cadenas de números de tres dígitos que representan la dirección de un equipo en la red inalámbrica. Esto significa que cad